# lab 12-1. RNN basics
- http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
- http://learningtensorflow.com/index.html
- http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

TensorFlow 1.x selected.


In [ ]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False # 경고문구 무시하고 싶을 때

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

## RNN
![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)


In [ ]:
tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility
sess = tf.InteractiveSession()

In [ ]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [ ]:
with tf.variable_scope('one_cell') as scope:

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)

    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size) # num_units : output 지정
    # cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    print(cell.output_size, cell.state_size)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32) #  rnn 실행 -> 결과, 마지막 상태값

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.]]], dtype=float32)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

2 2
array([[[-0.5568914 ,  0.36737362]]], dtype=float32)


In [ ]:
# One cell LSTM input_dim (4) -> output_dim (2)

hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size) # num_units : output 지정
x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32) #  rnn 실행 -> 결과, 마지막 상태값

sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())

array([[[-0.05421947,  0.12440576]]], dtype=float32)


## Sequence_length
![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [ ]:
with tf.variable_scope('two_sequances') as scope:

    # One cell RNN input_dim (4) -> output_dim (2), sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.5126983 ,  0.5338934 ],
        [ 0.21498992,  0.42855003],
        [-0.6585775 , -0.49946102],
        [-0.12646362,  0.2815579 ],
        [-0.29461876, -0.50052476]]], dtype=float32)


In [ ]:
# One cell RNN input_dim (4) -> output_dim (2), sequence: 5
hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size) #, reuse=True)
x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
print(x_data.shape)
pp.pprint(x_data)

outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.1702299 , -0.10432763],
        [-0.03229115, -0.1704478 ],
        [-0.02036469, -0.13807331],
        [-0.01399429, -0.12083788],
        [ 0.05348274,  0.06013514]]], dtype=float32)


## Batch_size
- 여러개의 시퀀스
![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

In [ ]:
with tf.variable_scope('3_batches') as scope:

    # One cell RNN input_dim (4) -> output_dim (2), sequence: 5, batch: 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size) #, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-2.1446506e-02,  6.9402412e-02],
        [-4.2407191e-03, -7.8120954e-02],
        [-1.1915967e-01, -2.4526760e-01],
        [-2.0080103e-01, -3.6001003e-01],
        [-2.3785511e-01, -3.0602500e-01]],

       [[ 8.1154445e-05, -1.4305250e-01],
        [-1.4125080e-01, -7.8153379e-02],
        [-2.2641259e-01, -2.3875631e-01],
        [-2.6866373e-01, -3.5584775e-01],
        [-3.0290794e-01, -4.2929482e-01]],

       [[-1.1008195e-01, -1.8190321e-01],
        [-1.9152264e-01, -3.1767112e-01],
        [-1.2687388e-01, -3.5920212e-01],
        [-1.0973829e-01, -3.9237773e-01

## 다양한 case의 활용

In [ ]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    # initial_state setting
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.03644354,  0.11180278],
        [ 0.07228415,  0.2713655 ],
        [ 0.12513869,  0.02897254],
        [ 0.19417495, -0.11492663],
        [ 0.2042482 , -0.17052297]],

       [[ 0.11234799,  0.14210875],
        [-0.02058888, -0.02849266],
        [ 0.11455012, -0.1646588 ],
        [ 0.19579726, -0.23882318],
        [ 0.24232948, -0.28252   ]],

       [[ 0.12012594, -0.14313665],
        [ 0.19794056, -0.22549453],
        [ 0.3469109 , -0.04528597],
        [ 0.34520838,  0.13550226],
        [ 0.22662374, -0.03318373]]], dtype=float32)


In [ ]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

# 3*5*3 =45
x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)


In [ ]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 4.18571606e-02, -1.52896151e-01,  5.21332286e-02,
         -1.92335352e-01, -1.40673695e-02],
        [ 2.27333933e-01, -3.22266579e-01,  1.62473116e-02,
         -2.92620361e-01,  5.08655310e-01],
        [ 1.31769419e-01, -4.76887643e-01,  2.66776187e-03,
         -2.93687880e-01,  8.90565872e-01],
        [ 4.89410013e-02, -5.49484253e-01,  4.06011561e-04,
         -2.56443471e-01,  9.80457306e-01],
        [ 1.83269382e-02, -5.77668011e-01,  3.01327436e-05,
         -2.14854583e-01,  9.96516109e-01]],

       [[ 7.45152263e-03, -5.39832354e-01, -2.07130815e-05,
         -1.74139813e-03,  7.61142910e-01],
        [ 2.59211380e-03, -6.28490984e-01, -5.42672296e-06,
         -1.98460324e-03,  9.63898838e-01],
        [ 9.75349627e-04, -6.60204113e-01, -1.06814775e-06,
         -1.85423542e-03,  9.95025396e-01],
        [ 3.76582611e-04, -6.81766808e-01, -1.99162457e-07,
         -1.65254017e-03,  9.99322653e-01],
        [ 1.45971790e-04, -6.98877692e-01, -3.35479164e-08,
  

In [ ]:
with tf.variable_scope('bi-directional') as scope:

    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[ 4.57419753e-02, -1.21115334e-01,  7.00143948e-02,
          6.11512177e-02, -1.00029439e-01],
        [-1.01744495e-01, -3.25308114e-01,  7.89827853e-02,
          5.53445779e-02, -1.61796100e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00]],

       [[-6.63339436e-01, -2.85580635e-01,  9.76432930e-04,
          2.49305631e-05, -1.39329515e-07],
        [-6.83405399e-01, -2.92200506e-01,  2.37191023e-04,
          1.60219552e-05, -1.00622346e-08],
        [-7.13696897e-01, -2.64169693e-01,  6.70627123e-05,
          9.77656600e-06, -5.46056533e-10],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00

- MultiRNN

In [ ]:
[cell for i in range(3)], [cell] * 3

([<tensorflow.python.ops.rnn_cell_impl.LSTMCell at 0x7f27d72035f8>,
  <tensorflow.python.ops.rnn_cell_impl.LSTMCell at 0x7f27d72035f8>])

In [ ]:
def make_cell(lstm_size):
  return tf.nn.rnn_cell.BasicLSTMCell(lstm_size, state_is_tuple=True)

network = rnn.MultiRNNCell([make_cell(5) for _ in range(3)], 
                                state_is_tuple=True)
network

In [ ]:
with tf.variable_scope('MultiRNNCell') as scope:
    # Make rnn
    # cell = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True) 
    # cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True) # 3 layers <- 오류 발생, cell이 얕은 복사되어 공유되어 dim이 변경되는 문제
    cell = rnn.MultiRNNCell([make_cell(5) for _ in range(3)], 
                                state_is_tuple=True)
    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("MultiRNNCell/rnn/transpose_1:0", shape=(3, 5, 5), dtype=float32)
array([[[-0.00548773,  0.00031074, -0.00111628,  0.00269011,
          0.0065346 ],
        [-0.01737265, -0.00115935, -0.00254868,  0.0086645 ,
          0.01678029],
        [-0.02707977, -0.00427497, -0.00554896,  0.01448952,
          0.01917405],
        [-0.02694288, -0.00646928, -0.01246946,  0.01653825,
          0.00397717],
        [-0.01765722, -0.00690353, -0.0223918 ,  0.0154137 ,
         -0.02536287]],

       [[ 0.01470773,  0.00084936, -0.00423007, -0.00242023,
         -0.02024855],
        [ 0.03775766,  0.00293562, -0.01233814, -0.00446592,
         -0.05289685],
        [ 0.06198976,  0.00635233, -0.02299861, -0.00490806,
         -0.08824326],
        [ 0.0837485 ,  0.01091939, -0.03499368, -0.00368379,
         -0.12083056],
        [ 0.10185771,  0.01635823, -0.04734741, -0.00120809,
         -0.1485025 ]],

       [[ 0.01532483,  0.00114205, -0.00424368, -0.00299469,
        

### Cost - sequence_loss

In [ ]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32) # 1, 0, 1 로 예측

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.5967595


In [ ]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction4 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction4, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval(),
      "Loss4: ", sequence_loss4.eval())

Loss1:  0.5130153 Loss2:  0.3711007 Loss3:  1.3132616 Loss4:  0.64659494


## lab12-4: Stacked RNN + Softmax Layer 참고
- flatten for softmax/ FC layer


In [ ]:
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

pp.pprint(x_data) # batch_size=3, sequence_length=5, hidden_size(input_dim)=3
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)
array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.],
       [12., 13., 14.],
       [15., 16., 17.],
       [18., 19., 20.],
       [21., 22., 23.],
       [24., 25., 26.],
       [27., 28., 29.],
       [30., 31., 32.],
       [33., 34., 35.],
       [36., 37., 38.],
       [39., 40., 41.],
       [42., 43., 44.]], dtype=float32)
array([[[  25.,   28.,   31.,   34.,   37.],
        [  70.,   82.,   94.,  106.,  118.],
        [ 115.,  136.,  157.,  178.,  199.],
        [ 160.,  190.,  220.,  250.,  280.],
        [ 205.,  244.,  283.,  322.,  361.]],

 

## lab12-5: Dynamic RNN 참고
- Dynamic RNN
  - different sequence length
  - 주어진 문장(단어)의 길이가 제각각 다를 때 활용

In [ ]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (2), batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    # sequence_length : [5,3,4]
    # -> using hello, eol__, llee_ (_ : padding과 같음), loss 계산에 영향을 주지 않기 위함
    # 실제로는 길이가 다른 문장, 단어를 학습할 때 사용할 수 있음
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.0103391 ,  0.00308083],
        [ 0.1391541 ,  0.11371334],
        [ 0.01963985,  0.18396455],
        [-0.03370879,  0.22584783],
        [-0.15720497,  0.21307617]],

       [[ 0.14740118,  0.11249505],
        [-0.01537107,  0.09415425],
        [-0.05253058,  0.14720133],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.04119788,  0.09633451],
        [-0.07079215,  0.16184852],
        [ 0.07851066,  0.2089067 ],
        [ 0.19451506,  0.2465296 ],
        [ 0.        ,  0.        ]]], dtype=float32)


In [ ]:
with tf.variable_scope('dynamic_rnn') as scope:
    cell = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose_1:0", shape=(3, 5, 5), dtype=float32)
array([[[ 1.2965105e-01,  6.3837811e-02,  1.5848710e-01,  1.3336371e-01,
         -4.3612695e-03],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]],

       [[ 6.3341960e-02,  6.1804406e-10,  5.2389449e-01,  1.0072193e-02,
          2.6724557e-03],
        [ 1.1050750e-01,  1.3841002e-11,  5.5535597e-01,  4.4731353e-03,
          9.2576357e-04],
        [ 1.4224447e-01,  2.5133343e-13,  5.7187390e-01,  2.0495669e-03,
          3.4949076e-04],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+